In [1]:
# curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
# curl https://packages.microsoft.com/config/debian/11/prod.list | tee /etc/apt/sources.list.d/mssql-release.list
# apt-get update
# apt update
# pip install pyodbc
# apt install -y curl gnupg apt-transport-https unixodbc-dev jq postgresql-client
# ACCEPT_EULA=Y apt install -y msodbcsql18
# odbcinst -q -d | grep "ODBC Driver 18"
# pip install anthropic
# pip install tqdm

In [2]:
import pyodbc
import pandas as pd

from tqdm import tqdm
from io import StringIO
from datetime import datetime

In [3]:
# MCP server main API

def get_substring_between(text, delimiter, occurrence=1):
    """
    Returns the substring between the N-th and (N+1)-th occurrence of a delimiter.
    
    Parameters:
        text (str): The string to search in.
        delimiter (str): The delimiter to search for.
        occurrence (int): Which occurrence to consider (default is 1, i.e., between first and second delimiter).
    
    Returns:
        str: Substring between the delimiters, or None if not found.
    """
    parts = text.split(delimiter)
    if len(parts) > occurrence:
        return parts[occurrence]
    return None

def parse_markdown_table_no_unnamed(md_table: str) -> pd.DataFrame:
    """
    Parse a Markdown-style table into a pandas DataFrame without creating 'Unnamed' columns.
    """
    # Split lines and remove the separator line
    lines = [line for line in md_table.strip().splitlines() if not line.startswith('|---')]
    
    # Split each line by '|' and strip whitespace
    table_data = []
    for line in lines:
        # Remove leading/trailing pipes, then split
        row = [cell.strip() for cell in line.strip().strip('|').split('|')]
        table_data.append(row)
    
    # First row is headers
    headers = table_data[0]
    data = table_data[1:]
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=headers)
    df = drop_unnamed_columns(df)

    return df

def drop_unnamed_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drops all columns from the DataFrame whose name contains 'Unnamed'.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        
    Returns:
        pd.DataFrame: DataFrame without columns containing 'Unnamed'.
    """
    # Keep only columns that do NOT contain 'Unnamed'
    df_clean = df.loc[:, ~df.columns.str.contains('^Unnamed', case=False)]
    return df_clean

def Connect_DB(username, password, server, port, database_name):
    try:
        # Connect to the SQL Server
        conn = pyodbc.connect(
        f'DRIVER={{ODBC Driver 18 for SQL Server}};'
        f'SERVER={server};DATABASE={database_name};'
        f'UID={username};PWD={password};;Encrypt=yes;TrustServerCertificate=yes;'
        f'TlsVersion=1.1'
    )

        # cursor = conn.cursor()
        # cursor.execute("select top 1 * from mcp_guests where e_mail = 'adossantos@purgatory.ski'")  # Test query

        # # Print results
        # for row in cursor.fetchall():
        #     print(row)

        # conn.close()
        print("Connection successful!")

    except Exception as e:
        print("Error connecting to database:", e)

    return conn

def Map_DB_tables_and_columns(conn):
    """
    Returns a DataFrame with all tables and columns in the database.
    
    Parameters:
        conn : pyodbc.Connection
            An open connection to the MSSQL database.
            
    Returns:
        pd.DataFrame with columns: 'table', 'column_name'
    """
    query = """
    SELECT TABLE_NAME AS [table], COLUMN_NAME AS column_name
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_CATALOG = DB_NAME()  -- current database
    ORDER BY TABLE_NAME, ORDINAL_POSITION;
    """
    
    df = pd.read_sql(query, conn)
    # Returns df with columns: [table, column_name]

    return df

def DB_Close(conn):
    conn.close()

def Execute_Query(conn, QUERY):
    
    cursor = conn.cursor()

    cursor.execute(QUERY)
    rows = cursor.fetchall()
    column_values = [row[0] for row in rows]

    return rows

def Check_Field_Is_Key(conn, Table_Name, Column_Name, THR=.98):
    # This function checks whether a given column in a table in a DB connected with conn is a primary key or not.abs
    # To do that it uses a THR of how many unique values are there in the column relative to it length (should be all unique)
    # THR = .98 - The percentage that the len of unique values should be of the total len of the column

    Field_Is_Key = 0
    
    # Get data from database
    cursor = conn.cursor()

    QUERY = f"SELECT {Column_Name} FROM {Table_Name}"
    RESULT = Execute_Query(conn, QUERY)

    # Check unique values
    RESULT_SET = set(RESULT)
    RATIO = len(RESULT_SET) / len(RESULT)
    if RATIO>=THR:
        Field_Is_Key = 1

    return Field_Is_Key

def Check_Table_Depedencies(conn, DB_Mapping_DF=None):
    # This function runs through all tables of the database connected with conn and returns a DF of dependencies between tables
    # via primary key (example: ID in a table)

    if DB_Mapping_DF is None:
        DB_Mapping_DF = Map_DB_tables_and_columns(conn)
        Table_Names = DB_Mapping_DF['table'].unique()

    Primary_Key_Columns_DICT = {}
    Primary_Key_Columns_Counter = 0

    print('Analyzing table columes...')

    for ii, table_name in enumerate(Table_Names):
        table_columns = list(DB_Mapping_DF[DB_Mapping_DF['table']==table_name]['column_name'])

        for jj, column_name in enumerate(table_columns):
            print(f'Analyzing table No.{ii+1} out of {len(Table_Names)} - column No.{jj+1} out of {len(table_columns)}', end='\r')

            if Check_Field_Is_Key(conn, table_name, column_name):
                try:
                    Primary_Key_Columns_DICT[table_name].append(column_name)
                except:
                    Primary_Key_Columns_DICT[table_name] = []
                    Primary_Key_Columns_DICT[table_name].append(column_name)

                Primary_Key_Columns_Counter += 1

    print('\nDONE.')
    print(f'{Primary_Key_Columns_Counter}\tPrimary key columns were found in {len((Primary_Key_Columns_DICT))} tables')

    return Table_Names

def pyodbc_rows_to_dataframe(cursor):
    rows = cursor.fetchall()
    if not rows:
        return pd.DataFrame()
    columns = [col[0] for col in cursor.description]
    data = [tuple(row) for row in rows]  # ← critical conversion
    return pd.DataFrame(data, columns=columns)


In [4]:
# Connect to DB
username = "shakudo"
password = "6?jsV4Mb{&1)q34v"
server = "63.158.251.204"  # host,port
port = 1433
database_name = "siriusware"  # if known

conn = Connect_DB(username, password, server, port, database_name)

Connection successful!


In [5]:
# # Example query: select all rows from a table
# Table_Name = 'access'
# query = f"SELECT * FROM {Table_Name}"

# cursor.execute(query)

# # Fetch all results
# rows = cursor.fetchall()

# print(f'{len(rows)} Rows retrieved')


In [6]:
# Stored procedures

STORED_PROC_DICT = {}
STORED_PROC_DICT['Revenue'] = 'exec Shakudo_DMRGetRevenue @database=?, @group_no=?, @date_ini=?, @date_end=?'
STORED_PROC_DICT['Payroll'] = 'exec Shakudo_DMRGetPayroll @resort=?, @date_ini=?, @date_end=?'
STORED_PROC_DICT['Visits'] = 'exec Shakudo_DMRGetVists @resort=?, @date_ini=?, @date_end=?'
STORED_PROC_DICT['Weather'] = 'exec Shakudo_GetSnow @resort=?, @date_ini=?, @date_end=?'

# -- @resort = Snowbowl, Purgatory, Brian Head, Lee Canyon, Nordic Valley, Sipapu, Willammette
# -- @date_ini/@date_end please pass with time: example for yesterday: @date_ini = '2025-10-10', @date_end = '2025-10-10 23:59:59'

In [5]:
# Run a stored procedure

# Create a cursor and execute the stored procedure
cursor = conn.cursor()

database = 'Purgatory'
group_no = 46
date_ini = datetime(2025, 3, 10, 0, 0, 0)
date_end = datetime(2025, 3, 10, 23, 59, 59)
# date_ini = '2025-03-10'
# date_end = '2025-30-10 23:59:59'
resort = 'Purgatory'

# cursor.execute(STORED_PROC_DICT['Revenue'], (database, group_no, date_ini, date_end))
# cursor.execute(STORED_PROC_DICT['Payroll'], (resort, date_ini, date_end))
# cursor.execute(STORED_PROC_DICT['Visits'], (resort, date_ini, date_end))
# cursor.execute(STORED_PROC_DICT['Weather'], (resort, date_ini, date_end))

# cursor.execute("exec Shakudo_DMRGetRevenue @database=?, @group_no=?, @date_ini=?, @date_end=?", (database, group_no, date_ini, date_end))
# cursor.execute("exec Shakudo_DMRGetPayroll @resort=?, @date_ini=?, @date_end=?", (resort, date_ini, date_end))
cursor.execute("exec Shakudo_DMRGetVists @resort=?, @date_ini=?, @date_end=?", (resort, date_ini, date_end))
# cursor.execute("exec Shakudo_GetSnow @resort=?, @date_ini=?, @date_end=?", (resort, date_ini, date_end))

# 3. Fetch results
rows = cursor.fetchall()

for row in rows:
    print(row)

print(f'len(rows) = {len(rows)}')

# 4. Clean up
cursor.close()


('Purgatory', datetime.date(2025, 3, 10), 14, 'Brian Passes')
('Purgatory', datetime.date(2025, 3, 10), 1, 'Nordic Passes')
('Purgatory', datetime.date(2025, 3, 10), 9, 'Pajarito Passes')
('Purgatory', datetime.date(2025, 3, 10), 668, 'Purgatory Comp Tickets')
('Purgatory', datetime.date(2025, 3, 10), 62, 'Purgatory Emp Passes')
('Purgatory', datetime.date(2025, 3, 10), 866, 'Purgatory Passes')
('Purgatory', datetime.date(2025, 3, 10), 1894, 'Purgatory Tickets')
('Purgatory', datetime.date(2025, 3, 10), 8, 'Sandia Passes')
('Purgatory', datetime.date(2025, 3, 10), 1, 'Sipapu Passes')
('Purgatory', datetime.date(2025, 3, 10), 18, 'Snowbowl Emp Passes')
('Purgatory', datetime.date(2025, 3, 10), 213, 'Snowbowl Passes')
len(rows) = 11


In [8]:
QUERY = """
declare @database nvarchar(20) = 'Snowbowl' -- Snowbowl, Brian, Lee Canyon, Purgatory, MCP, Sipapu, Nordic
declare @group_no int = -1 -- necessary for shared databases: 
-- Database: Purgatory 
--46	*PURGATORY               
--54	*HESPERUS                
--59	*SNOWCAT                 
--67	*SPIDER MOUNTAIN         
--70	*DMMA                    
--71	*WILLAMETTE              
-- Database: MCP
--9	** PAJARITO              
--10	** SANDIA                
--12	** WILLAMETTE            
--13	** AZ SNOWBOWL (temporary, only used week of 4th July 2025)

declare @date_ini datetime = '2024-03-05'
declare @date_end datetime = '2025-07-05 23:59:59'

SELECT
d.title as DepartmentTitle,
r.*
FROM
(
	select 
	p.user_code as account, 
	p.user_code2 as department, 
	--p.user_code3 as item, 
	sum(revenue) as revenue
	from
	(
		select t.resort, 
		t.pr_ctr_1 as pr_ctr_no,
		sum(pcsplit_1) as revenue
		from transact t
		where date_time between @date_ini and @date_end
		and department <> '**TRANS**'
		and t.resort = @database
		and t.pcsplit_1 <> 0
		and (
				@group_no = -1 OR 
				(t.salespoint in (select salespoint from sp_link where resort = @database and group_no = @group_no))
			)
		group by t.resort, t.pr_ctr_1

		union all
		
		select t.resort, 
		t.pr_ctr_2 as pr_ctr_no,
		sum(pcsplit_2) as revenue
		from transact t
		where date_time between @date_ini and @date_end
		and department <> '**TRANS**'
		and t.resort = @database
		and t.pcsplit_2 <> 0
		and (
				@group_no = -1 OR 
				(t.salespoint in (select salespoint from sp_link where resort = @database and group_no = @group_no))
			)
		group by t.resort, t.pr_ctr_2

		union all
		
		select t.resort, 
		t.pr_ctr_3 as pr_ctr_no,
		sum(pcsplit_3) as revenue
		from transact t
		where date_time between @date_ini and @date_end
		and department <> '**TRANS**'
		and t.resort = @database
		and t.pcsplit_3 <> 0
		and (
				@group_no = -1 OR 
				(t.salespoint in (select salespoint from sp_link where resort = @database and group_no = @group_no))
			)
		group by t.resort, t.pr_ctr_3
		
		union all
		
		select t.resort, 
		t.pr_ctr_4 as pr_ctr_no,
		sum(pcsplit_4) as revenue
		from transact t
		where date_time between @date_ini and @date_end
		and department <> '**TRANS**'
		and t.resort = @database
		and t.pcsplit_4 <> 0
		and (
				@group_no = -1 OR 
				(t.salespoint in (select salespoint from sp_link where resort = @database and group_no = @group_no))
			)
		group by t.resort, t.pr_ctr_4
		
		union all
		
		select t.resort, 
		t.pr_ctr_5 as pr_ctr_no,
		sum(pcsplit_5) as revenue
		from transact t
		where date_time between @date_ini and @date_end
		and department <> '**TRANS**'
		and t.resort = @database
		and t.pcsplit_5 <> 0
		and (
				@group_no = -1 OR 
				(t.salespoint in (select salespoint from sp_link where resort = @database and group_no = @group_no))
			)
		group by t.resort, t.pr_ctr_5
		
		union all
		
		select t.resort, 
		t.pr_ctr_6 as pr_ctr_no,
		sum(pcsplit_6) as revenue
		from transact t
		where date_time between @date_ini and @date_end
		and department <> '**TRANS**'
		and t.resort = @database
		and t.pcsplit_6 <> 0
		and (
				@group_no = -1 OR 
				(t.salespoint in (select salespoint from sp_link where resort = @database and group_no = @group_no))
			)
		group by t.resort, t.pr_ctr_6
	) t
	left join prof_ctr p on t.resort = p.resort and t.pr_ctr_no = p.pr_ctr_no
	where p.user_code >= '40000' and p.user_code <= '49999'
	group by p.user_code, p.user_code2 -- , p.user_code3 (item)
	having sum(revenue) <> 0
) r
left join intacct.dbo.Department d on d.DepartmentId = r.department
"""


In [9]:

# Render the full tables for MCP (Alex email)
DB_Map_Enable = 0
if DB_Map_Enable:
    print('Getting all tables and their column names into DF...', end='')
    Tables_Columns_MAPPING_df = Map_DB_tables_and_columns(conn)
    print(Tables_Columns_MAPPING_df.head(5))
    print('DONE')

RESULT = Execute_Query(conn, QUERY)
for ii, item in enumerate(RESULT):
    print(f'{ii}\t{item}')

print(f'len(RESULT) = {len(RESULT)}')
# df.to_csv('MCP_Datalake_Mapping.CSV')
# print(df)

pass

0	('Tickets', '40100                    ', 'D1500                    ', Decimal('16398693.2500'))
1	('Tickets', '48105                    ', 'D1500                    ', Decimal('-85673.7200'))
2	('Tickets', '41660                    ', 'D1500                    ', Decimal('136.8500'))
3	('General Administration', '41800                    ', 'D8010                    ', Decimal('512.0000'))
4	('Hart Prairie Retail', '40400                    ', 'D4032                    ', Decimal('2408668.6000'))
5	('Agassiz Restaurant', '40500                    ', 'D5110                    ', Decimal('2296225.1800'))
6	('Base Camp Restaurant', '40500                    ', 'D5112                    ', Decimal('487169.2000'))
7	('Fort Valley Lodge', '40500                    ', 'D4034                    ', Decimal('3689.2900'))
8	('Fremont Restaurant', '40500                    ', 'D5113                    ', Decimal('347544.6600'))
9	('Events', '41800                    ', 'D5810                    

In [10]:
# Close database connection
DB_Close(conn)